In [1]:
import numpy as np
import h5py
import os

import multiprocessing
from scipy.misc import imread
from scipy.misc import imresize

In [19]:
batch_size = 100000
image_size = 128
num_cpus = multiprocessing.cpu_count()

def process(f):
    global image_size
    im = imread(f, mode='RGB')
    im = imresize(im, (image_size, image_size), interp='bicubic')
    return im


In [20]:
%%time
## Train
prefix = '/home/ILSVRC2012/train/'
l = list(map(lambda x : os.path.join(prefix, x), os.listdir(prefix)))

i = 0
imagenet = np.zeros((len(l), image_size, image_size, 3), dtype='uint8')
pool = multiprocessing.Pool(num_cpus)
while i < len(l):
    current_batch = l[i:i + batch_size]    
    current_res = np.array(pool.map(process, current_batch))
    imagenet[i:i + batch_size] = current_res    
    i += batch_size
    print(i, 'images')

100000 images
200000 images
300000 images
400000 images
500000 images
600000 images
700000 images
800000 images
900000 images
1000000 images
1100000 images
1200000 images
1300000 images
CPU times: user 11.8 s, sys: 15.5 s, total: 27.3 s
Wall time: 12min 38s


In [21]:
%%time
# Val
prefix = '/home/ILSVRC2012/val/'
l_val = list(map(lambda x : os.path.join(prefix, x), os.listdir(prefix)))

i = 0
imagenet_val = np.zeros((len(l_val), image_size, image_size, 3), dtype='uint8')
pool = multiprocessing.Pool(multiprocessing.cpu_count())

while i < len(l_val):
    current_batch = l_val[i:i + batch_size]    
    current_res = np.array(pool.map(process, current_batch))
    imagenet_val[i:i + batch_size] = current_res    
    i += batch_size
    print(i, 'images')

100000 images
CPU times: user 528 ms, sys: 8.7 s, total: 9.23 s
Wall time: 39.6 s


In [22]:
with h5py.File('/home/aroyer/Datasets/imagenet-128.hdf5', 'w') as f:
    f['train'] = imagenet
    f['val'] = imagenet_val